<a href="https://colab.research.google.com/github/SaharMalaika/CODING-SAMURAI-INTERNSHIP-TASK/blob/main/Chatbot_using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
pip install nltk

In [45]:
import nltk
nltk.download('punkt')  ##package with pretrained tokenizer
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [46]:
def tokenize(sentence):
  return nltk.word_tokenize(sentence)








In [47]:
def stem(word):
  return stemmer.stem(word.lower())

In [48]:
import numpy as np
def bag_of_words(tokenized_sentence, all_words):
  pass

  tokenized_sentence = [stem(w) for w in tokenized_sentence]
  bag = np.zeros(len(all_words), dtype=np.float32)
  for idx, w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1.0   ##giving 1 to index where the word exists

  return bag



In [49]:

import json
import nltk

with open('intents.json','r') as f:
  intents=json.load(f)


all_words=[]
tags=[]
xy=[]
for intent in intents['intents']:
  tag=intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w=tokenize(pattern)
    all_words.extend(w)
    xy.append((w,tag))
  ignore_words=['?',',','.','!']
  all_words=[stem(w) for w in all_words if w not in ignore_words]
  all_words=sorted(set(all_words))  ##removing duplicate elements
  tags=sorted(set(tags))
print(tags)



['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


In [50]:
##creating bag of words
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

X_train= []
y_train= []
for (pattern_sentence,tag) in xy:
  bag=bag_of_words(pattern_sentence,all_words)
  X_train.append(bag)
  label=tags.index(tag)
  y_train.append(label) ## cross entropy loss

X_train=np.array(X_train)
y_train=np.array(y_train)




In [51]:
class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples=len(X_train)
    self.x_data=X_train
    self.y_data=y_train

  #dataset[index]
  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]

  def __len__(self):
    return self.n_samples


# Hyper-parameters
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)


dataset=ChatDataset()
train_loader=DataLoader(dataset=dataset,batch_size=8,shuffle=True,num_workers=0)





In [52]:
import torch
import torch.nn as nn
#from model import NeuralNet


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):   ##bag of words as input
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [53]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [54]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [55]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

Epoch [100/1000], Loss: 1.2844
Epoch [200/1000], Loss: 0.0692
Epoch [300/1000], Loss: 0.0467
Epoch [400/1000], Loss: 0.0076
Epoch [500/1000], Loss: 0.0026
Epoch [600/1000], Loss: 0.0043
Epoch [700/1000], Loss: 0.0010
Epoch [800/1000], Loss: 0.0135
Epoch [900/1000], Loss: 0.0020
Epoch [1000/1000], Loss: 0.0016
final loss: 0.0016
training complete. file saved to data.pth


In [57]:
import random
import json
import torch





In [58]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as f:
    intents = json.load(f)

In [59]:
FILE = "data.pth"
data = torch.load( FILE)

<ipython-input-59-a234400205f2>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load( FILE)


In [60]:
input_size= data["input_size"]
hidden_size=data["hidden_size"]
output_size=data["output_size"]
all_words=data["all_words"]
tags=data["tags"]
model_state=data["model_state"]

In [61]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


NeuralNet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [63]:
bot_name= "SAM"
print("Let's chat! type 'quit' to exit")
while True:
  sentence=input("You: ")
  if sentence=="quit":
    break

  sentence=tokenize(sentence)
  X=bag_of_words(sentence,all_words)
  X=X.reshape(1,X.shape[0])
  X=torch.from_numpy(X)

  output=model(X)
  _,predicted=torch.max(output,dim=1)
  tag=tags[predicted.item()]

  probs=torch.softmax(output,dim=1)
  prob=probs[0][predicted.item()]

  if prob.item()>0.75:
    for intent in intents["intents"]:
      if tag==intent["tag"]:
        print(f"{bot_name}: {random.choice(intent['responses'])}")
  else:
    print(f"{bot_name}: I do not understand...")

  for intent in intents["intents"]:
    if tag==intent["tag"]:
      print(f"{bot_name}: {random.choice(intent['responses'])}")


Let's chat! type 'quit' to exit
You: hi
SAM: Hi there, how can I help?
SAM: Hey :-)
You: what do u sell
SAM: We have coffee and tea
SAM: We sell coffee and tea
You: how long takes shipping
SAM: Delivery takes 2-4 days
SAM: Shipping takes 2-4 days
You: quit
